## Бибилиотеки

In [80]:
# !pip install --force-reinstall    llama-cloud==0.1.35 \
#     llama-cloud-services==0.6.53 \
#     llama-index==0.12.52 \
#     llama-index-agent-openai==0.4.12 \
#     llama-index-cli==0.4.4 \
#     llama-index-core==0.12.52.post1 \
#     llama-index-embeddings-openai==0.3.1 \
#     llama-index-indices-managed-llama-cloud==0.8.0 \
#     llama-index-instrumentation==0.4.0 \
#     llama-index-llms-openai==0.4.7 \
#     llama-index-llms-openai-like==0.4.0 \
#     llama-index-llms-together==0.3.2 \
#     llama-index-multi-modal-llms-openai==0.5.3 \
#     llama-index-program-openai==0.3.2 \
#     llama-index-question-gen-openai==0.3.1 \
#     llama-index-readers-file==0.4.11 \
#     llama-index-readers-llama-parse==0.4.0 \
#     llama-index-workflows==1.2.0 \
#     llama-parse==0.6.53

In [81]:
import os, json, random
from llama_index.llms.openai_like import OpenAILike
from llama_index.core.tools import FunctionTool
from llama_index.core.agent import ReActAgent
from llama_index.core.llms import ChatMessage


from dotenv import load_dotenv

In [82]:
load_dotenv()
API_KEY = os.getenv("TOGETHER_API_KEY")

# Модель
MODEL_NAME = 'meta-llama/Llama-3.3-70B-Instruct-Turbo'

llm = OpenAILike(
    model=MODEL_NAME,
    api_base="https://api.together.xyz/v1",
    api_key=API_KEY,
    is_chat_model=True,
    is_function_calling_model=True,
    temperature=0.3
)

In [83]:
SYSTEM_PROMPT = '''Вы — специализированный агент производственной системы. Ваша основная задача — обрабатывать запросы, связанные с определённой областью деятельности завода, и использовать доступные инструменты только тогда, когда это необходимо для выполнения конкретной операционной задачи.

Вы должны:
1. Анализировать входящий запрос на предмет его отношения к вашей предметной области: {DOMAIN_DESCRIPTION}.
2. Если запрос попадает в вашу зону ответственности и требует получения данных или выполнения действия — использовать соответствующий инструмент.
3. Если запрос не связан с вашей областью, является общим, теоретическим, бытовым или не требует вызова инструмента — отвечать кратко и нейтрально, **не пытаясь использовать инструменты**.
4. Никогда не выдумывать факты, не генерировать предположения о данных, которые можно получить через инструмент, если он ещё не был вызван.
5. Сохранять формально-деловой тон, избегать избыточных пояснений и отклонений от темы.

{ADDITIONAL_INSTRUCTIONS}

Ваша цель — обеспечить точность, эффективность и уместность ответов в рамках вашей функции на производстве. Все операционные данные доступны только через инструменты. Не отвечайте за другие отделы. Не комбинируйте информацию из других доменов.'''.strip()

## Генерация данных

In [84]:
# === Создаём папку data, если её нет ===
DATA_DIR = 'data'
os.makedirs(DATA_DIR, exist_ok=True)

# === Файлы с тестовыми данными (в папке data/) ===
LOGISTICS_FILE = os.path.join(DATA_DIR, 'logistics.json')
PRODUCTION_FILE = os.path.join(DATA_DIR, 'production.json')
QUALITY_FILE = os.path.join(DATA_DIR, 'quality.json')
PRICES_FILE = os.path.join(DATA_DIR, 'prices.json')
STOCK_FILE = os.path.join(DATA_DIR, 'stock.json')

# # === Файлы с тестовыми данными ===
# LOGISTICS_FILE = 'logistics.json'
# PRODUCTION_FILE = 'production.json'
# QUALITY_FILE = 'quality.json'
# PRICES_FILE = 'prices.json'
# STOCK_FILE = 'stock.json'

# === Генерация тестовых данных ===
def generate_mock_data():
    # Закупщик
    with open(PRICES_FILE, 'w', encoding='utf-8') as f:
        json.dump([
            {'product':'Example Product','firm':'Competitor A','price':random.randint(100,200)},
            {'product':'Example Product','firm':'Competitor B','price':random.randint(100,200)}
        ], f, ensure_ascii=False, indent=2)
    with open(STOCK_FILE, 'w', encoding='utf-8') as f:
        json.dump([{'product_id':'12345','quantity':random.randint(0,50)}], f, ensure_ascii=False, indent=2)

    # Логистика: поставки сырья
    with open(LOGISTICS_FILE, 'w', encoding='utf-8') as f:
        json.dump([
            {
                'supplier': 'RawMaterial Inc.',
                'product': 'Пластик ПЭ-300',
                'delivery_date': '2025-04-10',
                'status': 'в пути',
                'tracking_id': 'TRK123456'
            },
            {
                'supplier': 'SteelGlobal',
                'product': 'Сталь Ст3',
                'delivery_date': '2025-04-05',
                'status': 'доставлено',
                'tracking_id': 'TRK789012'
            }
        ], f, ensure_ascii=False, indent=2)

    # Производство: загрузка линий
    with open(PRODUCTION_FILE, 'w', encoding='utf-8') as f:
        json.dump([
            {
                'line_id': 'LINE-A1',
                'product': 'Тостер Модель X',
                'status': 'работает',
                'output_per_hour': 120,
                'defect_rate': 1.2
            },
            {
                'line_id': 'LINE-B2',
                'product': 'Чайник Эко 2000',
                'status': 'в ремонте',
                'output_per_hour': 0,
                'defect_rate': 0.0
            }
        ], f, ensure_ascii=False, indent=2)

    # Контроль качества: инциденты и проверки
    with open(QUALITY_FILE, 'w', encoding='utf-8') as f:
        json.dump([
            {
                'batch_id': 'BATCH-2025-04-01',
                'product': 'Фен TurboDry',
                'inspection_date': '2025-04-01',
                'result': 'прошёл',
                'defects_found': 2,
                'certified': True
            },
            {
                'batch_id': 'BATCH-2025-04-02',
                'product': 'Миксер PowerMix',
                'inspection_date': '2025-04-02',
                'result': 'не прошёл',
                'defects_found': 15,
                'certified': False,
                'reason': 'нарушение изоляции'
            }
        ], f, ensure_ascii=False, indent=2)

# Генерируем тестовые данные
generate_mock_data()

## Агент - Закупщик

In [85]:
%%capture
# Инструменты
def fetch_competitor_prices(product_name: str) -> str:
    """Получает цены конкурентов для указанного продукта"""
    data = json.load(open(PRICES_FILE, 'r', encoding='utf-8'))
    results = [f"{e['firm']}: ${e['price']}" for e in data if e['product'] == product_name]
    return "\n".join(results) if results else "Не найдено данных о ценах"

def check_stock(product_id: str) -> str:
    """Проверяет количество товара на складе по ID"""
    data = json.load(open(STOCK_FILE, 'r', encoding='utf-8'))
    for e in data:
        if e['product_id'] == product_id:
            return f"На складе: {e['quantity']} шт."
    return "Товар не найден"

# Создание инструментов
tool_prices = FunctionTool.from_defaults(
    name="fetch_competitor_prices",
    description="Получает информацию о ценах конкурентов для указанного продукта",
    fn=fetch_competitor_prices
)

tool_stock = FunctionTool.from_defaults(
    name="check_stock",
    description="Проверяет количество товара на складе по идентификатору",
    fn=check_stock
)

DOMAIN_DESCRIPTION = "мониторинг рыночных цен конкурентов и управление складскими запасами"
ADDITIONAL_INSTRUCTIONS = "Для запросов о ценах используйте fetch_competitor_prices с названием продукта. Для проверки наличия применяйте check_stock с идентификатором товара. На все прочие запросы отвечайте строго: 'Извините, данная функция не входит в список моих обязанностей. Я могу выполнять только: 1) проверку цен конкурентов по названию продукта; 2) проверку наличия товара на складе по идентификатору.' Запрещено обрабатывать запросы вне указанных функций или генерировать информацию без данных инструментов."

agent_buyer = ReActAgent.from_tools(
    [tool_prices, tool_stock],
    llm=llm,
    verbose=True,
    system_prompt=SYSTEM_PROMPT.format(
        DOMAIN_DESCRIPTION=DOMAIN_DESCRIPTION,
        ADDITIONAL_INSTRUCTIONS=ADDITIONAL_INSTRUCTIONS,
    )
)

### Тестовые кейсы

In [86]:
# # Примеры использования
# print("\nПример 1: Проверка цен")
# print(call_agent("Какие цены на Example Product у конкурентов?"))

# print("\nПример 2: Проверка склада")
# print(call_agent("Сколько товара с ID 12345 на складе?"))

# print("\nПример 3: Комбинированный запрос")
# print(call_agent("Сначала узнай цены на Example Product у конкурентов, потом проверь склад для ID 12345"))

## Агент - Логистика

In [87]:
%%capture
# Инструменты для Логистики
def track_delivery(tracking_id: str) -> str:
    """Отслеживает статус поставки по трек-номеру"""
    data = json.load(open(LOGISTICS_FILE, 'r', encoding='utf-8'))
    for item in data:
        if item['tracking_id'] == tracking_id:
            return (f"Поставка {tracking_id}:\n"
                    f"Поставщик: {item['supplier']}\n"
                    f"Товар: {item['product']}\n"
                    f"Дата доставки: {item['delivery_date']}\n"
                    f"Статус: {item['status']}")
    return "Поставка не найдена"

def get_upcoming_deliveries() -> str:
    """Возвращает список предстоящих поставок"""
    data = json.load(open(LOGISTICS_FILE, 'r', encoding='utf-8'))
    upcoming = [d for d in data if d['status'] == 'в пути']
    if not upcoming:
        return "Нет активных поставок."
    result = "Предстоящие поставки:\n"
    for d in upcoming:
        result += f"- {d['product']} от {d['supplier']}, до {d['delivery_date']} (ID: {d['tracking_id']})\n"
    return result

# Логистика
tool_track_delivery = FunctionTool.from_defaults(
    fn=track_delivery,
    name="track_delivery",
    description="Отслеживает статус поставки по трек-номеру"
)

tool_upcoming_deliveries = FunctionTool.from_defaults(
    fn=get_upcoming_deliveries,
    name="get_upcoming_deliveries",
    description="Возвращает список предстоящих поставок сырья"
)

DOMAIN_DESCRIPTION = "управление поставками сырья и материалов: отслеживание статуса доставок, сроков прибытия, данных о поставщиках и логистических операциях"
ADDITIONAL_INSTRUCTIONS = "При запросах о статусе поставки используйте данные о трек-номерах и сроках. Если поставка задерживается, сообщайте только факт из данных. Не предлагайте альтернативы и не прогнозируйте последствия."

# Агент-логист
agent_logistics = ReActAgent.from_tools(
    [tool_track_delivery, tool_upcoming_deliveries],
    llm=llm,
    verbose=True,
    system_prompt=SYSTEM_PROMPT.format(
        DOMAIN_DESCRIPTION=DOMAIN_DESCRIPTION,
        ADDITIONAL_INSTRUCTIONS=ADDITIONAL_INSTRUCTIONS,
    )
)

## Агент - Производства

In [88]:
%%capture
# Инструменты для Производства
def get_line_status(line_id: str) -> str:
    """Возвращает статус производственной линии"""
    data = json.load(open(PRODUCTION_FILE, 'r', encoding='utf-8'))
    for line in data:
        if line['line_id'] == line_id:
            return (f"Линия {line_id}:\n"
                    f"Продукт: {line['product']}\n"
                    f"Состояние: {line['status']}\n"
                    f"Выпуск: {line['output_per_hour']} шт/час\n"
                    f"Брак: {line['defect_rate']}%")
    return "Линия не найдена"

def get_production_summary() -> str:
    """Общая информация о производстве"""
    data = json.load(open(PRODUCTION_FILE, 'r', encoding='utf-8'))
    total_lines = len(data)
    active_lines = len([l for l in data if l['status'] == 'работает'])
    result = f"Всего линий: {total_lines}, активных: {active_lines}\n\nДетали:\n"
    for line in data:
        result += f"{line['line_id']} - {line['product']} ({line['status']})\n"
    return result

# Производство
tool_line_status = FunctionTool.from_defaults(
    fn=get_line_status,
    name="get_line_status",
    description="Возвращает текущий статус производственной линии по ID"
)

tool_production_summary = FunctionTool.from_defaults(
    fn=get_production_summary,
    name="get_production_summary",
    description="Показывает общий статус производства: сколько линий работает"
)

DOMAIN_DESCRIPTION = "контроль состояния производственных линий, загрузки оборудования, объёмов выпуска и уровня простоев"
ADDITIONAL_INSTRUCTIONS = "Фокусируйтесь на текущем статусе линий: работает, в ремонте, остановлена. При запросах о производительности указывайте выпуск в штуках в час и уровень брака. Не объясняйте причины простоев, если они не указаны в данных."

# Агент-производства
agent_production = ReActAgent.from_tools(
    [tool_line_status, tool_production_summary],
    llm=llm,
    verbose=True,
    system_prompt=SYSTEM_PROMPT.format(
        DOMAIN_DESCRIPTION=DOMAIN_DESCRIPTION,
        ADDITIONAL_INSTRUCTIONS=ADDITIONAL_INSTRUCTIONS,
    )
)

## Агент - Контроль качества

In [89]:
%%capture
# Инструменты для Контроля Качества
def check_batch_quality(batch_id: str) -> str:
    """Проверяет результаты контроля качества по номеру партии"""
    data = json.load(open(QUALITY_FILE, 'r', encoding='utf-8'))
    for batch in data:
        if batch['batch_id'] == batch_id:
            status = "✅ Сертифицировано" if batch['certified'] else "❌ Не прошло контроль"
            reason = f"\nПричина: {batch['reason']}" if not batch['certified'] else ""
            return (f"Партия {batch_id} ({batch['product']}):\n"
                    f"Результат: {batch['result']}\n"
                    f"Найдено дефектов: {batch['defects_found']}\n"
                    f"{status}{reason}")
    return "Партия не найдена"

def get_failed_batches() -> str:
    """Возвращает список партий, не прошедших контроль"""
    data = json.load(open(QUALITY_FILE, 'r', encoding='utf-8'))
    failed = [b for b in data if b['result'] == 'не прошёл']
    if not failed:
        return "Все партии прошли контроль."
    result = "Партии, не прошедшие контроль:\n"
    for b in failed:
        result += f"- {b['batch_id']} ({b['product']}): {b['reason']}\n"
    return result

# Качество
tool_check_batch = FunctionTool.from_defaults(
    fn=check_batch_quality,
    name="check_batch_quality",
    description="Проверяет результаты контроля качества по номеру партии"
)

tool_failed_batches = FunctionTool.from_defaults(
    fn=get_failed_batches,
    name="get_failed_batches",
    description="Возвращает список партий, не прошедших контроль качества"
)

DOMAIN_DESCRIPTION = "контроль качества выпускаемой продукции, проверка результатов тестирования партий, анализ выявленных дефектов и статуса сертификации"
ADDITIONAL_INSTRUCTIONS = "При проверке партии указывайте результат контроля, количество дефектов и причину отказа, если она есть. Если партия не прошла контроль, явно сообщайте об этом. Не давайте рекомендаций по устранению дефектов."

# Агент-качества
agent_quality = ReActAgent.from_tools(
    [tool_check_batch, tool_failed_batches],
    llm=llm,
    verbose=True,
    system_prompt=SYSTEM_PROMPT.format(
        DOMAIN_DESCRIPTION=DOMAIN_DESCRIPTION,
        ADDITIONAL_INSTRUCTIONS=ADDITIONAL_INSTRUCTIONS,
    )
)

In [90]:
def call_agent(agent, query: str) -> str:
    """Функция для взаимодействия с любым агентом"""
    response = agent.chat(query)
    return str(response)

## Оркестратор агентов

In [91]:
ORCHESTRATOR_SYSTEM_PROMPT = """Вы — **Оркестратор агентов**, центральный координатор, управляющий несколькими специализированными агентами:

1. **Агент-закупщик** — отвечает за цены конкурентов и наличие на складе.
2. **Агент-логист** — отвечает за статус поставок, сроки, трек-номера.
3. **Агент-производства** — отвечает за статус линий, выпуск, простои.
4. **Агент-качества** — отвечает за результаты проверки партий, брак, сертификацию.

### Ваши правила:
- Внимательно анализируйте запрос пользователя.
- Используйте **только те инструменты (агенты), которые необходимы**.
- Если вопрос требует данных из нескольких доменов — вызовите несколько агентов.
- После получения всех данных — **объедините ответ в один связный, лаконичный ответ**.
- Говорите на языке пользователя (обычно русский), избегайте технических терминов, если не требуется.
- Не вымышляйте информацию. Всегда полагайтесь на ответы агентов.
- Не объясняйте, как вы работали — просто дайте ответ.

Пример:
> Пользователь: Почему задерживается выпуск продукции?
> Вы: Проверяю статус поставок компонентов и состояние производственной линии...
> (вызов agent_logistics и agent_production)
> Ответ: Поставка компонентов задерживается на 2 дня. Линия 4 простаивает в ожидании.
""".strip()

In [92]:
%%capture

def query_buyer_agent(query: str) -> str:
    """Передаёт запрос агенту-закупщику."""
    response = agent_buyer.chat(query)
    return str(response)

def query_logistics_agent(query: str) -> str:
    """Передаёт запрос агенту-логисту."""
    response = agent_logistics.chat(query)
    return str(response)

def query_production_agent(query: str) -> str:
    """Передаёт запрос агенту-производства."""
    response = agent_production.chat(query)
    return str(response)

def query_quality_agent(query: str) -> str:
    """Передаёт запрос агенту-качества."""
    response = agent_quality.chat(query)
    return str(response)

# --- Создание инструментов ---
tool_buyer = FunctionTool.from_defaults(
    fn=query_buyer_agent,
    name="agent_buyer",
    description="Используйте, если вопрос о ценах конкурентов или наличии на складе."
)

tool_logistics = FunctionTool.from_defaults(
    fn=query_logistics_agent,
    name="agent_logistics",
    description="Используйте, если вопрос о статусе поставок, доставках, трек-номерах."
)

tool_production = FunctionTool.from_defaults(
    fn=query_production_agent,
    name="agent_production",
    description="Используйте, если вопрос о статусе линий, объёме выпуска, простоях."
)

tool_quality = FunctionTool.from_defaults(
    fn=query_quality_agent,
    name="agent_quality",
    description="Используйте, если вопрос о контроле качества, браке, результатах проверки партий."
)

In [93]:
%%capture
# --- Создаём оркестратор ---
orchestrator = ReActAgent.from_tools(
    tools=[
        tool_buyer,
        tool_logistics,
        tool_production,
        tool_quality,
    ],
    llm=llm,
    verbose=True,
    system_prompt=ORCHESTRATOR_SYSTEM_PROMPT,
)

## Цензор

In [94]:
# Системный промпт — строгий фильтр
system_prompt = '''
Ты — фильтр запросов. Определи, относится ли вопрос к деловой сфере производства:
- Производственные процессы
- Управление поставками, логистика
- Закупки сырья и материалов
- Контроль качества, выпуск продукции
- Работа оборудования, простои

Если запрос относится к этим темам — верни **исходный вопрос без изменений**.
Если НЕ относится — верни **точно**: "Запрос не относится к деловой сфере производства."

Правила:
- Никаких пояснений, комментариев, вежливостей.
- Не используй форматирование, кавычки, маркеры.
- Не добавляй "Ответ:", "Результат:" и т.п.
- Возвращай ТОЛЬКО один из двух вариантов:
      1. Исходный вопрос (дословно)
      2. "Запрос не относится к деловой сфере производства."

Примеры:
Вопрос: Сколько стоит тонна алюминия у поставщиков?
Ответ: Сколько стоит тонна алюминия у поставщиков?

Вопрос: Как приготовить кофе?
Ответ: Запрос не относится к деловой сфере производства.
'''.strip()

In [95]:
def check_message(text):
    # Создаем сообщения
    messages = [
        ChatMessage(role="system", content=system_prompt),
        ChatMessage(role="user", content=text)
    ]

   # Вызываем chat
    response = llm.chat(messages)
    return response.message.blocks[0].text

In [96]:
# !pip install -U llama-index

## Запуск пайплайна

In [97]:
s = input('Введите запрос для ахуенных агеннтов')
response = check_message(s)
print(s)

Сколько стоит тонна алюминия у поставщиков?


In [98]:
if response == 'Запрос не относится к деловой сфере производства.':
    print('Запрос ошибочен, уточните запрос.')
else:
    print('- ' * 10)
    print(orchestrator.chat(s))

- - - - - - - - - - 
> Running step e5921c52-4951-42de-8303-2bc545101bcd. Step input: Сколько стоит тонна алюминия у поставщиков?
Thought: The current language of the user is: Russian. I need to use a tool to help me answer the question.
Action: agent_buyer
Action Input: {'query': 'current price of aluminum from suppliers'}
> Running step 442ee925-989b-474b-9bfb-c14cbf2da372. Step input: current price of aluminum from suppliers
Thought: The current language of the user is: English. I need to use a tool to help me answer the question. However, I realize that the provided tools are not suitable for getting the current price of aluminum from suppliers, as they are related to competitor prices and stock checking.
Answer: I cannot answer the question with the provided tools.
Observation: I cannot answer the question with the provided tools.
> Running step 9daec78b-210b-446f-998b-4b065d76bc88. Step input: None
Thought: I need to try a different approach to answer the question about the price